# 04: Decider Pattern — Event-Sourced Aggregates

Models a **fulfillment** bounded context with two aggregates using the [Decider pattern](https://thinkbeforecoding.com/post/2021/12/17/functional-event-sourcing-decider):

| Aggregate | Commands | Events |
|-----------|----------|--------|
| **Order** | PlaceOrder, CancelOrder | OrderPlaced, OrderCancelled |
| **Shipment** | SchedulePickup, ConfirmDelivery | PickupScheduled, DeliveryConfirmed |

Each aggregate has:
- `commands.ts` — command types and factory functions
- `decider.ts` — `decide` (state + command → events) and `evolve` (state + event → state)

KindScript enforces:
- **Purity** — aggregate code has no I/O imports (deciders are pure functions)
- **Isolation** — aggregates cannot import from each other
- **Overlap detection** — sibling members can't share files (auto-checked)
- **Exhaustiveness** — opt-in: every file must belong to a member

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
import { ksc, tree, copyFixture, writeFile, readFile, cleanup } from './lib.ts';
console.log("Setup complete.");

---

## Part 1: Build the Aggregates

We start with shared domain types, then build each aggregate with its commands and decider.

In [ ]:
const DEMO = copyFixture("decider");

writeFile(DEMO, "src/types.ts", `
export type Command = { readonly type: string };
export type Event = { readonly type: string; readonly timestamp: string };
`);

console.log("Wrote src/types.ts — shared Command and Event base types");

### Order Aggregate

Handles order lifecycle. `PlaceOrder` creates an order; `CancelOrder` cancels a placed order.

In [ ]:
writeFile(DEMO, "src/order/commands.ts", `
import type { Command } from '../types';

export type PlaceOrder = Command & {
  readonly type: 'PlaceOrder';
  readonly orderId: string;
  readonly customerId: string;
  readonly items: readonly string[];
};

export type CancelOrder = Command & {
  readonly type: 'CancelOrder';
  readonly orderId: string;
  readonly reason: string;
};

export type OrderCommand = PlaceOrder | CancelOrder;

export function placeOrder(
  orderId: string, customerId: string, items: string[]
): PlaceOrder {
  return { type: 'PlaceOrder', orderId, customerId, items };
}

export function cancelOrder(orderId: string, reason: string): CancelOrder {
  return { type: 'CancelOrder', orderId, reason };
}
`);

console.log("Wrote src/order/commands.ts — PlaceOrder, CancelOrder");

In [ ]:
writeFile(DEMO, "src/order/decider.ts", `
import type { Event } from '../types';
import type { OrderCommand } from './commands';

// ── Events ──

export type OrderPlaced = Event & {
  readonly type: 'OrderPlaced';
  readonly orderId: string;
  readonly customerId: string;
  readonly items: readonly string[];
};

export type OrderCancelled = Event & {
  readonly type: 'OrderCancelled';
  readonly orderId: string;
  readonly reason: string;
};

export type OrderEvent = OrderPlaced | OrderCancelled;

// ── State ──

export type OrderState = {
  readonly status: 'initial' | 'placed' | 'cancelled';
  readonly orderId: string | null;
  readonly customerId: string | null;
  readonly items: readonly string[];
};

export const initialState: OrderState = {
  status: 'initial',
  orderId: null,
  customerId: null,
  items: [],
};

// ── Decide: (state, command) → events ──

function decideHandler(
  state: OrderState, command: OrderCommand
): OrderEvent[] {
  switch (command.type) {
    case 'PlaceOrder':
      if (state.status !== 'initial') return [];
      return [{
        type: 'OrderPlaced',
        orderId: command.orderId,
        customerId: command.customerId,
        items: command.items,
        timestamp: new Date().toISOString(),
      }];
    case 'CancelOrder':
      if (state.status !== 'placed') return [];
      return [{
        type: 'OrderCancelled',
        orderId: command.orderId,
        reason: command.reason,
        timestamp: new Date().toISOString(),
      }];
  }
}

export const decide = decideHandler;

// ── Evolve: (state, event) → state ──

function evolveHandler(
  state: OrderState, event: OrderEvent
): OrderState {
  switch (event.type) {
    case 'OrderPlaced':
      return {
        status: 'placed',
        orderId: event.orderId,
        customerId: event.customerId,
        items: event.items,
      };
    case 'OrderCancelled':
      return { ...state, status: 'cancelled' };
  }
}

export const evolve = evolveHandler;
`);

console.log("Wrote src/order/decider.ts — decide + evolve");

Each decider file follows the same shape:

```
function decideHandler(state, command) → events[]   // implementation
export const decide = decideHandler;                 // exported assignment

function evolveHandler(state, event) → state         // implementation
export const evolve = evolveHandler;                  // exported assignment
```

The function does the work; the const exports it. This separation keeps the implementation private and the public API clean.

### Shipment Aggregate

Handles shipping lifecycle. `SchedulePickup` dispatches a shipment; `ConfirmDelivery` marks it delivered.

In [ ]:
writeFile(DEMO, "src/shipment/commands.ts", `
import type { Command } from '../types';

export type SchedulePickup = Command & {
  readonly type: 'SchedulePickup';
  readonly shipmentId: string;
  readonly orderId: string;
  readonly address: string;
};

export type ConfirmDelivery = Command & {
  readonly type: 'ConfirmDelivery';
  readonly shipmentId: string;
  readonly signature: string;
};

export type ShipmentCommand = SchedulePickup | ConfirmDelivery;

export function schedulePickup(
  shipmentId: string, orderId: string, address: string
): SchedulePickup {
  return { type: 'SchedulePickup', shipmentId, orderId, address };
}

export function confirmDelivery(
  shipmentId: string, signature: string
): ConfirmDelivery {
  return { type: 'ConfirmDelivery', shipmentId, signature };
}
`);

console.log("Wrote src/shipment/commands.ts — SchedulePickup, ConfirmDelivery");

In [ ]:
writeFile(DEMO, "src/shipment/decider.ts", `
import type { Event } from '../types';
import type { ShipmentCommand } from './commands';

// ── Events ──

export type PickupScheduled = Event & {
  readonly type: 'PickupScheduled';
  readonly shipmentId: string;
  readonly orderId: string;
  readonly address: string;
};

export type DeliveryConfirmed = Event & {
  readonly type: 'DeliveryConfirmed';
  readonly shipmentId: string;
  readonly signature: string;
};

export type ShipmentEvent = PickupScheduled | DeliveryConfirmed;

// ── State ──

export type ShipmentState = {
  readonly status: 'pending' | 'in-transit' | 'delivered';
  readonly shipmentId: string | null;
  readonly orderId: string | null;
  readonly address: string | null;
};

export const initialState: ShipmentState = {
  status: 'pending',
  shipmentId: null,
  orderId: null,
  address: null,
};

// ── Decide: (state, command) → events ──

function decideHandler(
  state: ShipmentState, command: ShipmentCommand
): ShipmentEvent[] {
  switch (command.type) {
    case 'SchedulePickup':
      if (state.status !== 'pending') return [];
      return [{
        type: 'PickupScheduled',
        shipmentId: command.shipmentId,
        orderId: command.orderId,
        address: command.address,
        timestamp: new Date().toISOString(),
      }];
    case 'ConfirmDelivery':
      if (state.status !== 'in-transit') return [];
      return [{
        type: 'DeliveryConfirmed',
        shipmentId: command.shipmentId,
        signature: command.signature,
        timestamp: new Date().toISOString(),
      }];
  }
}

export const decide = decideHandler;

// ── Evolve: (state, event) → state ──

function evolveHandler(
  state: ShipmentState, event: ShipmentEvent
): ShipmentState {
  switch (event.type) {
    case 'PickupScheduled':
      return {
        status: 'in-transit',
        shipmentId: event.shipmentId,
        orderId: event.orderId,
        address: event.address,
      };
    case 'DeliveryConfirmed':
      return { ...state, status: 'delivered' };
  }
}

export const evolve = evolveHandler;
`);

console.log("Wrote src/shipment/decider.ts — decide + evolve");

In [ ]:
console.log("=== Project structure ===");
await tree(DEMO);

---

## Part 2: Architectural Definition

Now we add KindScript enforcement. The bounded context has two rules:
1. Each aggregate must be **pure** — no `fs`, `http`, or other I/O imports
2. Aggregates are **isolated** — order cannot import from shipment and vice versa

In [ ]:
writeFile(DEMO, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type Aggregate = Kind<"Aggregate", {}, { pure: true }>;

type FulfillmentBC = Kind<"FulfillmentBC", {
  order: [Aggregate, './order'];
  shipment: [Aggregate, './shipment'];
}, {
  noDependency: [["order", "shipment"], ["shipment", "order"]];
}>;

export const fulfillment = {
  order: {},
  shipment: {},
} satisfies Instance<FulfillmentBC, '.'>;
`);

console.log("Wrote src/context.ts");
console.log("");
console.log("  Aggregate      → pure (no I/O imports)");
console.log("  FulfillmentBC  → order and shipment cannot import from each other");

`Aggregate` is a leaf Kind with `{ pure: true }` — an intrinsic constraint meaning any folder mapped to an Aggregate must contain no I/O imports.

`FulfillmentBC` groups two Aggregate members with explicit tuple syntax declaring their locations:
- `order: [Aggregate, './order']` → `src/order/`
- `shipment: [Aggregate, './shipment']` → `src/shipment/`

The `noDependency` pairs forbid imports in both directions. `Instance<FulfillmentBC, '.'>` anchors the instance at `src/` (the directory containing `context.ts`).

The shared `src/types.ts` is in the instance root but not inside any member — both aggregates can import it freely.

---

## Part 3: Check — All Clean

In [ ]:
console.log("=== Check: should pass (no violations) ===");
await ksc("check", DEMO);

---

## Part 4: Catch Violations

### Aggregate isolation

What if order's decider needs shipment tracking info? Let's try importing from the shipment aggregate.

In [ ]:
const cleanOrderDecider = readFile(DEMO, "src/order/decider.ts");

writeFile(DEMO, "src/order/decider.ts",
  "import { initialState as shipmentState } from '../shipment/decider';\n" + cleanOrderDecider
);

console.log("=== Violation: order imports from shipment ===");
await ksc("check", DEMO);

`KS70001` — forbidden dependency. The order aggregate imported from the shipment aggregate, violating `noDependency: [["order", "shipment"]]`.

In a real system, if order needs shipment info, it should go through a domain event or an application-layer orchestrator — not a direct import.

In [ ]:
// Restore clean order decider
writeFile(DEMO, "src/order/decider.ts", cleanOrderDecider);

console.log("=== Fixed: restored clean order decider ===");
await ksc("check", DEMO);

### Purity

Deciders should be pure functions — no side effects. What if someone adds logging to a decider?

In [ ]:
const cleanShipmentDecider = readFile(DEMO, "src/shipment/decider.ts");

writeFile(DEMO, "src/shipment/decider.ts",
  "import * as fs from 'fs';\n" + cleanShipmentDecider
);

console.log("=== Violation: shipment decider imports 'fs' ===");
await ksc("check", DEMO);

`KS70003` — purity violation. The `Aggregate` Kind has `{ pure: true }`, so any file inside an Aggregate member must have zero I/O imports.

This catches the common mistake of adding `fs.writeFileSync` for debugging or `console.log` via `import * as console from 'console'` — the decider stays pure.

In [ ]:
// Restore clean shipment decider
writeFile(DEMO, "src/shipment/decider.ts", cleanShipmentDecider);

console.log("=== Fixed: restored clean shipment decider ===");
await ksc("check", DEMO);

---

## Part 5: Overlap Detection (KS70006)

KindScript automatically checks that sibling members don't share files. If two members point to overlapping directories, that's an architectural error — a file can't belong to two aggregates.

What if someone accidentally points both members to the same path?

In [ ]:
const cleanContext = readFile(DEMO, "src/context.ts");

// Break it: point both members to the same directory
writeFile(DEMO, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type Aggregate = Kind<"Aggregate", {}, { pure: true }>;

type FulfillmentBC = Kind<"FulfillmentBC", {
  order: [Aggregate, './order'];
  shipment: [Aggregate, './order'];
}>;

export const fulfillment = {
  order: {},
  shipment: {},
} satisfies Instance<FulfillmentBC, '.'>;
`);

console.log("=== Violation: order and shipment both map to ./order ===");
await ksc("check", DEMO);

`KS70006` — member overlap. Both `order` and `shipment` resolve to the same directory, so their file sets overlap completely. This is auto-detected — no constraint declaration needed. Every pair of sibling members is checked automatically.

Overlap detection catches copy-paste errors, misconfigured paths, and structural ambiguity where a file would belong to two members simultaneously.

In [ ]:
// Restore correct paths
writeFile(DEMO, "src/context.ts", cleanContext);

console.log("=== Fixed: restored correct member paths ===");
await ksc("check", DEMO);

---

## Part 6: Exhaustiveness — Unassigned Files (KS70007)

By default, files outside any member are silently allowed — `src/types.ts` sits in the instance root without belonging to `order` or `shipment`, and that's fine.

Adding `exhaustive: true` to constraints makes unassigned files an error. This is opt-in: enable it when you want full coverage.

In [ ]:
// Add exhaustive: true — all files in scope must belong to a member
writeFile(DEMO, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type Aggregate = Kind<"Aggregate", {}, { pure: true }>;

type FulfillmentBC = Kind<"FulfillmentBC", {
  order: [Aggregate, './order'];
  shipment: [Aggregate, './shipment'];
}, {
  noDependency: [["order", "shipment"], ["shipment", "order"]];
  exhaustive: true;
}>;

export const fulfillment = {
  order: {},
  shipment: {},
} satisfies Instance<FulfillmentBC, '.'>;
`);

console.log("=== Violation: types.ts is not in any member ===");
await ksc("check", DEMO);

`KS70007` — unassigned file. `src/types.ts` exists inside the instance scope (`src/`) but isn't inside any member directory (`src/order/` or `src/shipment/`). With `exhaustive: true`, this is an error.

The fix: either move shared types into a member, create a dedicated `shared` member for them, or remove `exhaustive: true` if full coverage isn't needed yet.

Default exclusions: `context.ts` files and test files (`*.test.ts`, `*.spec.ts`, `__tests__/`) are automatically excluded from exhaustiveness checks.

In [ ]:
// Fix: remove exhaustive (shared types are intentional here)
writeFile(DEMO, "src/context.ts", cleanContext);

console.log("=== Fixed: removed exhaustive constraint ===");
await ksc("check", DEMO);

In [ ]:
cleanup(DEMO);
console.log("Working directory cleaned up.");

---

## Summary

| Concept | Implementation |
|---------|---------------|
| Decider pattern | `decide(state, command) → events[]`, `evolve(state, event) → state` |
| Aggregate isolation | `noDependency` between order and shipment |
| Pure aggregates | `{ pure: true }` intrinsic on the Aggregate Kind |
| Shared types | `src/types.ts` in the instance root, accessible to both aggregates |
| Overlap detection | Auto-checked — catches mismatched member paths (KS70006) |
| Exhaustiveness | Opt-in `exhaustive: true` — catches unassigned files (KS70007) |

The decider pattern maps naturally to KindScript's enforcement:
- Each aggregate is a directory with a clean boundary
- Purity ensures deciders remain side-effect-free
- Isolation prevents aggregates from coupling to each other's internals
- Overlap detection catches misconfigured paths
- Exhaustiveness ensures every file is accounted for

**Previous:** [03-typekind.ipynb](03-typekind.ipynb) — declaration-level enforcement with TypeKind

**Reference:** [docs/03-constraints.md](../docs/03-constraints.md) — full constraint documentation